<a href="https://www.kaggle.com/aleksandrdubrovin/model-classification-training-snr-high?scriptVersionId=88379757" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Import libraries

In [ ]:
import zipfile
# from google.colab import files
import os
from sklearn.model_selection import train_test_split
import numpy as np
import keras
from keras import layers
from tensorflow.keras.utils import to_categorical
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.layers import Input, Dropout, Add, Dense, Reshape, Activation
from keras.layers import BatchNormalization, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam

## Getting and Loading the Data

In [ ]:
# Upload signals, labels, snrs
labels = np.load('/kaggle/input/deepsigio-radioml-201801a-new/labels.npy', mmap_mode = 'r')
signals = np.load('/kaggle/input/deepsigio-radioml-201801a-new/signals.npy', mmap_mode = 'r')
snrs = np.load('/kaggle/input/deepsigio-radioml-201801a-new/snrs.npy', mmap_mode = 'r')


In [ ]:
# Split arrays for two 3 parts (we take only third part of dataset of labeled signals because of the memory)
part = 3
signals = signals[::part, :, :]
labels = labels[::part, :]
snrs = snrs[::part, :]

print(signals.shape)
print(labels.shape)
print(snrs.shape)

In [ ]:
# Ndarray to array
snrs = np.ravel(snrs)
print(f"All possible SNRS: {np.unique(snrs)} db")

In [ ]:
# Masked numpy array 
c = np.ma.masked_where(snrs > 8, snrs)
msk = c.mask
# Count unique elements in array
print(np.unique(c.mask, return_counts=True))

In [ ]:
# Mask array of signals and labels (snrs > 8)
signals = signals[msk]
labels = labels[msk]

print(len(signals))

In [ ]:
# Train|test = 80|20

x_train,x_test, y_train, y_test = train_test_split(signals, labels, train_size=0.8, stratify=labels)

# print(f"Количество строк в y_train по классам: {np.bincount(y_train)}")
# print(f"Количество строк в y_test по классам: {np.bincount(y_test)}")
print(x_test.shape)
print(y_test.shape)

# Train|validation|test = 64|16|20
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, stratify=y_train)

# print(f"Количество строк в y_train по классам: {np.bincount(y_test)}")
# print(f"Количество строк в y_test по классам: {np.bincount(y_val)}")
print(x_train.shape)
print(y_train.shape)

## Create Residual Stack

In [ ]:
# Create residual and convolution blok
class Residual_block:
    kernel_size = 3
    strides = 1
    padding = 'same'
    data_format = "channels_last"

    def __init__(self, x, x_shortcut, filters):
        self.x = x
        self.filters = filters
        self.x_shortcut = x_shortcut

    def unit(self):
        x = Conv1D(self.filters, self.kernel_size, self.strides, self.padding, self.data_format)(self.x)
        x = Activation('relu')(x)
        x = Conv1D(self.filters, self.kernel_size, self.strides, self.padding, self.data_format)(x)
        x = Activation('linear')(x)
        # add skip connection
        if x.shape[1:] == self.x_shortcut.shape[1:]:
            x = Add()([x, self.x_shortcut])
        else:
            raise Exception('Skip Connection Failure!')
        return x

class Convolution_block:
    kernel_size = 1
    strides = 1
    padding = 'same'
    data_format = "channels_last"

    def __init__(self, x, filters):
        self.x = x
        self.filters = filters

    def unit(self):
        x = Conv1D(self.filters, self.kernel_size, self.strides, self.padding, self.data_format)(self.x)
        x = Activation('linear')(x)
        return x

In [ ]:
# Create residual stack
def residual_stack(x, filters):
    x = Convolution_block(x, filters)
    x = x.unit()
    
    x_shortcut = x
    x = Residual_block(x, x_shortcut, filters)
    x = x.unit()
    x_shortcut = x
    x = Residual_block(x, x_shortcut, filters)  
    x = x.unit()
    
    # max pooling layer
    x = MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last')(x)
#     print('Residual stack created')
    return x

## Define ResNet Model

In [ ]:
# define resnet model
def ResNet(input_shape, classes):   
    # create input tensor
    x_input = Input(input_shape)
    x = x_input
    # residual stack
    num_filters = 40
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    
    # output layer
    x = Flatten()(x)
    x = Dense(128, activation="selu", kernel_initializer="he_normal")(x)
    x = Dropout(.5)(x)
    x = Dense(128, activation="selu", kernel_initializer="he_normal")(x)
    x = Dropout(.5)(x)
    x = Dense(classes , activation='softmax', kernel_initializer = glorot_uniform(seed=0))(x)
    
    # Create model
    model = Model(inputs = x_input, outputs = x)
#     print('Model ResNet created')
    return model

## Option to Save Model Weights and History

In [ ]:
# option to save model weights and model history
save_model = False
save_history = False

# create directory for model weights
if save_model is True:
    weights_path = input("Name model weights directory: ")
    weights_path = "data/weights/" + weights_path

    try:
        os.mkdir(weights_path)
    except OSError:
        print ("Creation of the directory %s failed" % weights_path)
    else:
        print ("Successfully created the directory %s " % weights_path)
    print('\n')
    

# create directory for model history
if save_history is True:
    history_path = input("Name model history directory: ")
    history_path = "data/model_history/" + history_path

    try:
        os.mkdir(history_path)
    except OSError:
        print ("Creation of the directory %s failed" % history_path)
    else:
        print ("Successfully created the directory %s " % history_path)
    print('\n')

## Set Model Parameters

In [ ]:
# initialize optimizer 
adm = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# set number of epochs
num_epochs = int(input('Enter number of epochs: '))

# set batch size
batch = 32

# configure weights save

if save_model is True:
    filepath= weights_path + "/{epoch}.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode="auto")
    callbacks_list = [checkpoint]
else:
    callbacks_list = []

## Print Model Summary and Train Network

In [ ]:
# initialize and train model
model = ResNet((1024, 2), 24)
model.compile(optimizer=adm, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(x_train, y_train, epochs = num_epochs, batch_size = batch, callbacks=callbacks_list, validation_data=(x_val, y_val))

## Save Model History

In [ ]:
# record model history
train_accuracy = history.history['accuracy']
train_loss = history.history['loss']
val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']

if save_history is True:
    # save model history: loss and accuracy
    np.save(history_path + '/train_acc.npy', train_accuracy)
    np.save(history_path + '/train_loss.npy', train_loss)
    np.save(history_path + '/val_acc.npy', val_accuracy)
    np.save(history_path + '/val_loss.npy', val_loss)
    print("Model History Saved!")
    print('\n')

In [ ]:
model.save("model_high_SNR.h5")
print("Saved model to disk")

## Evaluate Model on Test Data

In [ ]:
# evaluate model on test data
loss, acc = model.evaluate(x_test, y_test, batch_size=32)
print('EVALUATING MODEL ON TEST DATA:')
print('Test Accuracy: ', str(round(acc*100, 2)), '%')
print('\n')